In [1]:
import datetime as dt

import panel as pn
import pandas as pd

import hvplot.pandas
hvplot.extension('bokeh')

pn.config.theme = 'dark'


def get_series(**options):
    df = pd.read_csv(options["url"], index_col=0)
    df.index = pd.to_datetime(df.index, utc=True).tz_convert(options["tz"])
    df = df.sort_index(ascending=True)
    return df[[options["column"]]].rename(columns={options["column"]: options["name"]})


def get_dataframe(**options):
    df = pd.read_csv(options["url"], index_col=0)
    df.index = pd.to_datetime(df.index, utc=True).tz_convert(options["tz"])
    df = df.sort_index(ascending=True)
    return df


active_tools = []
backend_opts={"plot.toolbar.autohide": True}
height=350
width=750

start_time = dt.datetime.now(dt.UTC) - dt.timedelta(days=7)
start_time

datetime.datetime(2024, 5, 21, 23, 42, 43, 944706, tzinfo=datetime.timezone.utc)

In [2]:
options = dict(
    url="https://raw.githubusercontent.com/ryanfobel/ontario-grid-data/main/data/clean/gridwatch.ca/hourly/summary.csv",
    tz="America/Toronto",
    column="",
)

df = get_dataframe(**options)
df = df[df.index > start_time]

In [3]:
plot_options = dict(
    value_label='MW',
    legend='bottom',
    title="Ontario grid supply and demand",
    height=height,
    width=width,
    stacked=False,    
    grid=True,
    alpha=0.5,
    hover=False,
)
columns = ['Power Generated (MW)', 'Ontario Demand (MW)', 'Imports (MW)', 'Exports (MW)', 'Net Import/Exports (MW)']
columns = {x: x.replace(" (MW)", "") for x in columns}
suply_demand = df[columns.keys()].rename(columns=columns).iloc[-24*7:].hvplot.area(**plot_options).opts(active_tools=active_tools, backend_opts=backend_opts)
suply_demand

:NdOverlay   [Variable]
   :Area   [index]   (MW)

In [4]:
plot_options = dict(
    value_label='MW',
    legend='bottom',
    title="Ontario grid generation",
    height=height,
    width=width,
    grid=True,
    stacked=True,
    alpha=0.5,
    ylim=(0, None),
    hover=False,
)

columns = {x: x.replace(" (MW)", "") for x in df.columns if x.endswith(" (MW)") and (x[0]==x[0].lower())}
generation = df[columns.keys()].rename(columns=columns).iloc[-24*7:].hvplot.area(**plot_options).opts(active_tools=active_tools, backend_opts=backend_opts)
generation

:NdOverlay   [Variable]
   :Area   [index]   (MW,Baseline)

In [5]:
plot_options = dict(
    value_label='%',
    legend='bottom',
    title="Ontario grid power mix (%)",
    height=height,
    width=width,
    grid=True,
    stacked=True,
    alpha=0.5,
    ylim=(0, 100),
    hover=False,
)

columns = {x: x.replace(" (%)", "") for x in df.columns if x.endswith(" (%)")}
generation_pct = df[columns.keys()].rename(columns=columns).iloc[-24*7:].hvplot.area(**plot_options).opts(active_tools=active_tools, backend_opts=backend_opts)
generation_pct

:NdOverlay   [Variable]
   :Area   [index]   (%,Baseline)

In [6]:
options_gridwatch = dict(
    name = "gridwatch",
    url="https://raw.githubusercontent.com/ryanfobel/ontario-grid-data/main/data/clean/gridwatch.ca/hourly/summary.csv",
    tz="America/Toronto",
    column = "CO2e Intensity (g/kWh)"
)

options_co2signal = dict(
    name="co2signal",
    url="https://raw.githubusercontent.com/ryanfobel/ontario-grid-data/main/data/clean/co2signal.com/CA-ON/hourly/output.csv",
    column = "data.carbonIntensity",
    tz="America/Toronto",
)

df = get_series(**options_gridwatch).join(
    get_series(**options_co2signal),
    how="inner"
)

plot_options = dict(
    value_label='g/kWh',
    legend='bottom',
    title="Ontario grid co2e intensity",
    height=height,
    width=width,
    grid=True,
    alpha=0.5,
    ylim=(0, None),
    hover=False,
)
df = df[df.index > start_time]
co2_intensity = df.hvplot.area(**plot_options).opts(active_tools=active_tools, backend_opts=backend_opts)
co2_intensity

:NdOverlay   [Variable]
   :Area   [index]   (g/kWh,Baseline)

In [7]:
template = pn.template.FastGridTemplate(
    row_height=200,
    theme_toggle=False,
    theme="dark",
    title="Ontario grid data",
    prevent_collision=True,
)
template.main[0:2,0:6]=suply_demand
template.main[0:2,6:12]=generation
template.main[2:4,0:6]=generation_pct
template.main[2:4,6:12]=co2_intensity

In [8]:
template.servable();

In [11]:
#!panel convert index.ipynb --to pyodide-worker --out ..\docs --pwa --title "Ontario grid data"

Successfully converted index.ipynb to pyodide-worker target and wrote output to index.html.
Successfully wrote icons and images.
Successfully wrote site.manifest.
Successfully wrote serviceWorker.js.


http://localhost:8000/docs

In [10]:
#!python -m http.server